<a href="https://colab.research.google.com/github/marceloc4rdoso/DesafioAI/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#O projeto explica todos os campos de um demonstrativo de pagamento, vulgo holerite. Que muitos trabaladores tem dúvidas sobre as informações contidas nesse documento.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import numpy as np
import pandas as pd

import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
DOCUMENT1 = {
    "title": "O que é Demosntrativo de Pagamento / Holerite / Contracheque",
    "content": "/content/sample_data/holerite.txt"}
DOCUMENT2 = {
    "title": "Cabeçalho do Demosntrativo de Pagamento / Holerite / Contracheque",
    "content": "/content/sample_data/header.txt"}
DOCUMENT3 = {
    "title": "Detalhe do Demosntrativo de Pagamento / Holerite / Contracheque",
    "content": "/content/sample_data/detail.txt"}
DOCUMENT4 = {
    "title": "Rodapé do Demosntrativo de Pagamento / Holerite / Contracheque",
    "content": "/content/sample_data/footer.txt"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Título", "Conteúdo"]
display(df)

In [ ]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                content=text,
                                task_type="RETRIEVAL_DOCUMENT",
                                title=title)["embedding"]

In [ ]:
model = 'models/embedding-001'

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
display(df)

In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
  embed_da_consulta = genai.embed_content(model=model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY")["embedding"]

  produto_escalares = np.dot(np.stack(df["Embeddings"]), embed_da_consulta)

  indice = np.argmax(produto_escalares)

  return df.iloc[indice]["Conteúdo"]

In [ ]:
consulta = input("Qual sua dúvida sobre Demonstrativos de Pagamento? ")

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

In [ ]:
generation_config = {
    "temperature": 0.8,
    "candidate_count": 1,
}

In [ ]:
prompt = f"Reescreva esse texto de uma forma descontratida, informal com sotaque gaúcho, sem informações que não façam parte do texto: {trecho}"
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)